In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split
import json

import glob
import random
import collections
from glob import glob
import numpy as np
import pandas as pd

#import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import shutil
import keras
from PIL import Image
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import json
import tensorflow as tf 
from keras.layers import Input
from keras import Sequential
from keras.layers import Dense, LSTM,Flatten, TimeDistributed, Conv2D, Dropout
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import re
import string

from keras.models import load_model
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D,Reshape, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten, UpSampling2D
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

from skimage.io import imread
from skimage.transform import resize
from keras import layers
from keras import models
import keras
from keras.layers import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

import tensorflow as tf
import tensorflow_addons as tfa
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Lambda, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Input
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
AUTO = tf.data.AUTOTUNE
from sklearn.metrics import confusion_matrix
import random

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 32
EPOCHS = 50
CROP_TO = 32
SEED = 26

PROJECT_DIM = 128
LATENT_DIM = 512
WEIGHT_DECAY = 0.0005
learning_rate = 0.0001
batch_size = 64
hidden_units = 512
projection_units = 256
num_epochs = 2
dropout_rate = 0.5

temperature = 0.05

C:\Users\Shaif\.conda\envs\myenv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pip install tensorflow-addons

In [2]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
#train_labels = to_categorical(train_labels)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255
test_images = test_images / 255 

x_train, x_unlab, y_train, y_unlab = train_test_split( train_images, train_labels , test_size=0.5, random_state=42 )
X_train, X_test, Y_train, Y_test = train_test_split( x_train,y_train , test_size=0.2, random_state=40 )

In [17]:
len(x_train)

50000

In [3]:
data_augmentation = keras.Sequential(
    [
        layers.experimental.preprocessing.Normalization(),
        layers.experimental.preprocessing.RandomFlip("horizontal"),
        layers.experimental.preprocessing.RandomRotation(0.02),
        layers.experimental.preprocessing.RandomWidth(0.2),
        layers.experimental.preprocessing.RandomHeight(0.2),
    ]
)

# Setting the state of the normalization layer.
data_augmentation.layers[0].adapt(X_train)

In [4]:


num_classes = 10
input_shape = (32, 32, 3)

def create_encoder():
    resnet = tf.keras.applications.ResNet50V2( include_top=False, weights=None, input_shape=input_shape, pooling="avg" )

    inputs = keras.Input(shape=input_shape)
    augmented = data_augmentation(inputs)
    outputs = resnet(augmented)
    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-encoder")
    return model

encoder = create_encoder()
encoder.summary()


Model: "cifar10-encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 sequential (Sequential)     (None, None, None, 3)     7         
                                                                 
 resnet50v2 (Functional)     (None, 2048)              23564800  
                                                                 
Total params: 23,564,807
Trainable params: 23,519,360
Non-trainable params: 45,447
_________________________________________________________________


In [5]:
def create_classifier(encoder, trainable=True):

    for layer in encoder.layers:
        layer.trainable = trainable

    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    features = layers.Dropout(dropout_rate)(features)
    features = layers.Dense(hidden_units, activation="relu")(features)
    features = layers.Dropout(dropout_rate)(features)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    model = keras.Model(inputs=inputs, outputs=outputs, name="cifar10-classifier")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(),
        metrics=[keras.metrics.SparseCategoricalAccuracy()],
    )
    return model

In [6]:
class SupervisedContrastiveLoss(keras.losses.Loss):
    def __init__(self, temperature=1, name=None):
        super(SupervisedContrastiveLoss, self).__init__(name=name)
        self.temperature = temperature

    def __call__(self, labels, feature_vectors, sample_weight=None):

        feature_vectors_normalized = tf.math.l2_normalize(feature_vectors, axis=1)

        logits = tf.divide( tf.matmul(  
            feature_vectors_normalized, tf.transpose(feature_vectors_normalized)),self.temperature,)
        
        return tfa.losses.npairs_loss(tf.squeeze(labels), logits)


def add_projection_head(encoder):
    
    inputs = keras.Input(shape=input_shape)
    features = encoder(inputs)
    outputs = layers.Dense(projection_units, activation="relu")(features)
    model = keras.Model(
        inputs=inputs, outputs=outputs, name="cifar-encoder_with_projection-head"
    )
    
    return model

In [11]:
encoder = create_encoder()

encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=SupervisedContrastiveLoss(temperature),
)

encoder_with_projection_head.summary()

history = encoder_with_projection_head.fit(
    x=X_train, y=Y_train, batch_size=batch_size, epochs=2)

Model: "cifar-encoder_with_projection-head"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 cifar10-encoder (Functional  (None, 2048)             23564807  
 )                                                               
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
Total params: 24,089,351
Trainable params: 24,043,904
Non-trainable params: 45,447
_________________________________________________________________
Epoch 1/2


KeyboardInterrupt: 

In [116]:
classifier = create_classifier(encoder, trainable=False) 

history = classifier.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=num_epochs) 

accuracy = classifier.evaluate(X_test, Y_test)[1] 
print(f"Test accuracy: {round(accuracy * 100, 2)}%") 

Epoch 1/2
157/157 [==============================] - 13s 47ms/step - loss: nan - sparse_categorical_accuracy: 0.1015
Epoch 2/2
157/157 [==============================] - 4s 21ms/step - loss: nan - sparse_categorical_accuracy: 0.1044
Test accuracy: 10.44%


In [14]:
# Helper function for merging new history objects with older ones
def append_history(losses, val_losses, accuracy, val_accuracy, history):
    losses = losses + history.history["loss"]
    val_losses = val_losses + history.history["val_loss"]
    accuracy = accuracy + history.history["accuracy"]
    val_accuracy = val_accuracy + history.history["val_accuracy"]
    return losses, val_losses, accuracy, val_accuracy


# Plotter function
def plot_history(losses, val_losses, accuracies, val_accuracies):
    plt.plot(losses)
    plt.plot(val_losses)
    plt.legend(["train_loss", "val_loss"])
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.show()

    plt.plot(accuracies)
    plt.plot(val_accuracies)
    plt.legend(["train_accuracy", "val_accuracy"])
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.show()

In [ ]:


def resmodel():
    
    
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), input_shape=(32, 32, 3)))
    model.add(layers.Conv2D(32, (3, 3), activation="relu"))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3),))
    model.add(layers.Conv2D(64, (3, 3),))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(128, (3, 3),))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(.2))
    model.add(layers.Flatten()) 
    model.add(layers.Dense(32))

    model.summary()
    return model

In [8]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
AUTO = tf.data.AUTOTUNE

def create_model(X,Y,X_test, Y_test,num_epochs):
    
    data_augmentation.layers[0].adapt(X)
    
    encoder = create_encoder()

    encoder_with_projection_head = add_projection_head(encoder)
    
    encoder_with_projection_head.compile(optimizer=keras.optimizers.Adam(learning_rate),
                                     loss=SupervisedContrastiveLoss(temperature),)
    
    
    history = encoder_with_projection_head.fit(x=X, y=Y, batch_size=256, epochs=num_epochs)
    
    
    classifier = create_classifier(encoder, trainable=False) 

    history = classifier.fit(x=X, y=Y, batch_size=batch_size, epochs=num_epochs) 

    accuracy = classifier.evaluate(X_test, Y_test)[1] 
    
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    #model.summary()
    return encoder,classifier

In [9]:
arr = np.concatenate((X_train, x_unlab))
X_all = arr
arr = np.concatenate((Y_train, y_unlab))
Y_all = arr

In [ ]:
# callbacks=[keras.callbacks.EarlyStopping(patience=4, verbose=1), ],

def train_full_model(X_train, X_test, Y_train, Y_test,num):
    
    X_train, Y_train = shuffle(X_train, Y_train)
    
    encoder,classifier = create_model(X_train,Y_train,X_test, Y_test,num)

    accuracy = classifier.evaluate(X_test, Y_test)[1] 
    print(f"Test accuracy: {round(accuracy * 100, 2)}%") 
    
    
    return encoder,classifier


encoder,classifier = train_full_model(X_all, X_test, Y_all, Y_test,1)

In [ ]:
encoder,classifier = train_full_model(X_train, X_test, Y_train, Y_test,5)

In [7]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

def train_active_learning_models(encoder,classifier,X_train,Y_train,x_unlab,y_unlab,X_test,Y_test,num_iterations,num_epochs=1):



    accuracy = classifier.evaluate(X_test, Y_test, batch_size=32)[1]
    
    
    print("Acc Test : ")
    print("\n")
    print(accuracy)
    print("\n")
    
    entropy = []
    l = len(y_unlab)
    d = int ( np.round ( l/num_iterations ) )
    
    for iteration in range(num_iterations):
        
        l = len(y_unlab)
        
        print("\n\n\n\n")
        
        print("Iteration : ")
        
        print(iteration+1)
        
        print("\n\n\n\n\n")
        nn_clusters = 10
        num_points_per_class = int ( np.round (d/10)   )
        
        budget =  int ( np.round (d/100)   )
        num_points_per_class = num_points_per_class - (budget)
        
        print("\n")
        print("Annotated in each iter : ")
        print(budget*20)
        print("\n")
        print("Chosen in each iter : ")
        print(num_points_per_class*10)
        print("\n")

        x_ulb = encoder.predict(x_unlab, batch_size=64)
        kmeans = KMeans(n_clusters=nn_clusters, init='k-means++', n_init=10).fit(x_ulb)

        closest_points_indices = []
        annotate_indices = []
        
        for i in range(10):
            
            cluster_center = kmeans.cluster_centers_[i]
            
            cluster_indices = np.where(kmeans.labels_ == i)[0]
            
            distances = np.linalg.norm(x_ulb[cluster_indices] - cluster_center, axis=1)
            
            closest_indices = cluster_indices[np.argsort(distances)[:num_points_per_class]]
            
            #annotate = cluster_indices[np.argsort(distances,-1)[:budget]] 
            
            closest_points_indices.extend(closest_indices)
            
            #annotate_indices.extend(annotate)

        chosen_indices = closest_points_indices
    
        rnd = chosen_indices
        
        if (iteration == 0):
            xx_ulb = classifier.predict(x_unlab, batch_size=32)
            argmax_indices = np.argmax(xx_ulb, axis=1)
            entropy = x_ulb[np.arange(len(xx_ulb)), argmax_indices]
            most_uncertain_idx = np.argsort(entropy)[-(budget*10):]
            
        else :
            xx_ulb = cls0.predict(x_unlab, batch_size=32)
            argmax_indices = np.argmax(xx_ulb, axis=1)
            entropy = x_ulb[np.arange(len(xx_ulb)), argmax_indices]
            
            xx_ulb = cls1.predict(x_unlab, batch_size=32)
            argmax_indices = np.argmax(xx_ulb, axis=1)
            entropy_new = x_ulb[np.arange(len(xx_ulb)), argmax_indices]
            
            most_uncertain_idx = np.argsort(np.abs(entropy_new - entropy))[::-1][:(budget*10)]
            
        annotate_indices.extend(most_uncertain_idx)
        annt = annotate_indices
        
        print(np.shape(rnd))
        print(np.shape(annotate_indices))
        print(np.shape(most_uncertain_idx))
        print(np.shape(annt))
        
        all = list(range(1, l))
        main_list = list(set(all) - set(rnd) - set(annt))
        new_lab = x_unlab[rnd]
        new_annt = x_unlab[annt]
        arr = np.concatenate((X_train,new_annt))
        X_train = arr

        
        accuracy = classifier.evaluate(new_lab, y_unlab[rnd], batch_size=32)[1]
        print("Acc : ")
        print("\n")
        print(accuracy)
        
        annt_y = y_unlab[annt]
        
        
        arr = np.concatenate((Y_train, annt_y))
        Y_train = arr
        
        
        x_unlab = x_unlab[main_list]

        y_unlab = y_unlab[main_list]
        
        #history = encoder_with_projection_head.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=5)
        
        print("Now2")
        print(np.shape(Y_train))
        print("\n")
        
        cls0=classifier
        cls1=classifier
        history = classifier.fit(x=X_train, y=Y_train, batch_size=16, epochs=5) 
        cls1=classifier
   
    
    return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
#train_labels = to_categorical(train_labels)
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255
test_images = test_images / 255 

x_train, x_unlab, y_train, y_unlab = train_test_split( train_images, train_labels , test_size=0.5, random_state=42 )
X_train, X_test, Y_train, Y_test = train_test_split( x_train,y_train , test_size=0.2, random_state=40 )

data_augmentation.layers[0].adapt(X_train)
encoder = create_encoder()
encoder_with_projection_head = add_projection_head(encoder)
encoder_with_projection_head.compile(optimizer=keras.optimizers.Adam(learning_rate),
                        loss=SupervisedContrastiveLoss(temperature))
    
history = encoder_with_projection_head.fit(x=X_train, y=Y_train, batch_size=64, epochs=5)
classifier = create_classifier(encoder, trainable=False) 
history = classifier.fit(x=X_train, y=Y_train, batch_size=64, epochs=5) 

encoder, classifier,X_train,Y_train,x_unlab,y_unlab = train_active_learning_models(encoder, classifier,X_train,Y_train,x_unlab,y_unlab,X_test,Y_test,num_iterations=7)

Epoch 1/5
313/313 [==============================] - 883s 3s/step - loss: 4.0011
Epoch 2/5
313/313 [==============================] - 883s 3s/step - loss: 3.8776
Epoch 3/5
313/313 [==============================] - 896s 3s/step - loss: 3.8134
Epoch 4/5
313/313 [==============================] - 904s 3s/step - loss: 3.7631
Epoch 5/5
313/313 [==============================] - 920s 3s/step - loss: 3.7228
Epoch 1/5
313/313 [==============================] - 87s 260ms/step - loss: 2.2893 - sparse_categorical_accuracy: 0.2341
Epoch 2/5
313/313 [==============================] - 88s 280ms/step - loss: 1.9369 - sparse_categorical_accuracy: 0.2792
Epoch 3/5
313/313 [==============================] - 84s 269ms/step - loss: 1.8383 - sparse_categorical_accuracy: 0.2986
Epoch 4/5
313/313 [==============================] - 86s 275ms/step - loss: 1.8147 - sparse_categorical_accuracy: 0.3096
Epoch 5/5
157/157 [==============================] - 17s 100ms/step - loss: 1.7134 - sparse_categorical_accurac

In [ ]:
def test(final_encoder, final_classifier, X_test, Y_test):
    print("-" * 100)
    print("Test set evaluation: ", model.evaluate( X_test, Y_test , verbose=0, return_dict=True), )
    print("-" * 100)

In [ ]:
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

def train_active_learning_models(encoder,classifier,X_train,Y_train,x_unlab,y_unlab,X_test,Y_test,num_iterations,num_epochs=1):
   
    
    entropy = []
    l = len(y_unlab)
    d = int ( np.round ( l/num_iterations ) )
    #x = int(np.round( l/d ))
    
    for iteration in range(num_iterations):
        
        l = len(y_unlab)
        
        print("\n\n\n\n")
        
        print("Iteration : ")
        
        print(iteration+1)
        
        print("\n\n\n\n\n")
        
        nn_clusters = 10
        num_points_per_class = int ( np.round (d/10)   )
        
        budget =  int(num_points_per_class /10 )
        num_points_per_class = num_points_per_class - budget
        
        print("\n")
        print("Annotated in each iter : ")
        print(budget*10)
        print("\n")
        print("Chosen in each iter : ")
        print(num_points_per_class*10)
        print("\n")
        
        budget = budget/2
  
        try :
            x_ulb = encoder.predict(x_unlab, batch_size=batch_size)
        except:
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        if (iteration == 0):
            argmax_indices = np.argmax(x_ulb, axis=1)
            entropy = x_ulb[np.arange(len(x_ulb)), argmax_indices]
            most_uncertain_idx = np.argsort(entropy)[-budget:]
            
        else :
            argmax_indices = np.argmax(x_ulb, axis=1)
            entropy_new = x_ulb[np.arange(len(x_ulb)), argmax_indices]
            most_uncertain_idx = np.argsort(np.abs(entropy_new - entropy))[::-1][:budget]
            entropy = entropy_new
        
        for i in range(10):
            
            cluster_center = kmeans.cluster_centers_[i]
            
            cluster_indices = np.where(kmeans.labels_ == i)[0]
            
            distances = np.linalg.norm(x_ulb[cluster_indices] - cluster_center, axis=1)
            
            #closest_indices = cluster_indices[np.argsort(distances)[:num_points_per_class]]
            
            annotate = cluster_indices[np.argsort(distances,-1)[:budget]] 
            
            #closest_points_indices.extend(closest_indices)
            
            annotate_indices.extend(annotate)
        
        print(np.shape(x_ulb))

        annotate_indices.extend(most_uncertain_idx)
        
    
        print(np.shape(annotate_indices))
        
        annt = annotate_indices
        
        
        all = list(range(1, l))
        main_list = list(set(all) - set(annt))
        
        #add those index to from unlablled set to training set
        #new_lab = x_unlab[rnd]
        new_annt = x_unlab[annt]
        arr = np.concatenate((X_train,new_annt))
        X_train = arr

        
        try :
            accuracy = classifier.evaluate(new_lab, y_unlab[rnd], batch_size=8)[1]
        except:
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        print("Acc : ")
        print("\n")
        print(accuracy)
        
        annt_y = y_unlab[annt]
        print("This")
        print(np.shape(Y_train))
        print("Now")
        #print(np.shape(new_y))
        
        #new_yy = np.argmax(new_y, axis=1)
        
        #new_yy = new_yy.reshape(len(new_y),1)
        
        
        arr = np.concatenate((Y_train, annt_y))
        Y_train = arr
        
        
        #create the new unlabelled set
        x_unlab = x_unlab[main_list]

        #create the new unlabelled label set
        y_unlab = y_unlab[main_list]
        
        
        #try :
            #history = encoder_with_projection_head.fit(x=X_train, y=Y_train, batch_size=batch_size, epochs=5)
        #except :
            #return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        print("Now2")
        print(np.shape(Y_train))
        print("\n")

        try :
            history = classifier.fit(x=X_train, y=Y_train, batch_size=4, epochs=1) 
        except:
            return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 
        
        #test on data
    
    #accuracy = classifier.evaluate(X_test, Y_test, batch_size=4)[1]
   
    
    return encoder, classifier,X_train,Y_train,x_unlab,y_unlab 


encoder, classifier,X_train,Y_train,x_unlab,y_unlab = train_active_learning_models(encoder, classifier,X_train,Y_train,x_unlab,y_unlab,X_test,Y_test,num_iterations=5)

In [32]:
Y_train

array([[8],
       [7],
       [8],
       ...,
       [3],
       [1],
       [3]], dtype=uint8)

In [33]:
a =np.argmax(Y_train)
a

20

In [30]:
a[1]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [16]:
np.shape(entropy)

(500,)

In [15]:
entropy = []
x_ulb = np.random.rand(500, 10)
d = 50
entropy = -np.sum(x_ulb * np.log2(x_ulb), axis=1)
entropy

array([3.20110315, 3.28177562, 4.28466595, 3.05597306, 3.33640414,
       3.98344707, 3.98666134, 4.06497428, 4.61018985, 3.41586384,
       3.54981722, 2.33302412, 4.17731431, 3.57913508, 3.18228556,
       3.16413198, 3.4941552 , 3.27640799, 3.24224851, 3.39185237,
       3.48132308, 3.75702406, 4.00681291, 4.2488176 , 3.35583782,
       3.56878115, 3.42452029, 2.61749758, 3.90703722, 4.2038775 ,
       3.1870036 , 3.83536252, 3.79608028, 4.51522581, 2.59527454,
       3.84691179, 2.88297182, 3.52732167, 2.96102925, 2.91282736,
       4.15999665, 3.5694131 , 4.14234101, 3.18152454, 3.96936405,
       3.16415649, 1.97201757, 2.94901682, 2.76512967, 3.55981656,
       3.66758179, 3.68152961, 3.10485252, 4.46079242, 3.82450346,
       3.19445297, 3.66166685, 3.86158568, 3.00918278, 3.70965738,
       3.98134712, 3.18750898, 4.04726307, 3.63988304, 2.82992511,
       4.16999716, 3.90547543, 3.96388523, 3.89829874, 2.95138845,
       3.50773932, 3.75178351, 3.43119484, 3.6526706 , 4.17225